# Full Experiment Rerun

Making a full rerun with LIME and SHAP with seeds and random state in the Explainer to be set along with

Initial runs had the following to reset seed
```
def set_seed(seed_value):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        # Optional: for determinism with CuDNN
        torch.backends.cudnn.deterministic = True
```


## Setup for Google Colab

Ran on T4 GPU

In [20]:
%pip install lime shap hf_xet

In [36]:
# sanity test

from lime.lime_text import LimeTextExplainer
import shap
import numpy as np
import pandas as pd
from transformers import pipeline, AutoTokenizer
import random
import torch
from typing import List, Union
from tqdm.notebook import tqdm


def set_seed(seed_value):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        # Optional: for determinism with CuDNN
        torch.backends.cudnn.deterministic = True



# Set seed for reproducibility
set_seed(42)


BATCH_SIZE = 512  # Adjust based on your GPU memory

# Load models and tokenizers once
pipe_cardiff = pipeline("text-classification",
                       model="cardiffnlp/twitter-roberta-base-hate-latest",
                       device=0 if torch.cuda.is_available() else -1,
                       batch_size=BATCH_SIZE)

tokenizer_cardiff = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")

# Create dataset class for more efficient processing
class HateSpeechDataset:
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]


# Unified prediction function for both models
def batch_predict(texts: Union[List[str], np.ndarray], pipeline_fn):
    """Run predictions in batches for efficiency"""
    if isinstance(texts, np.ndarray):
        texts = texts.tolist()

    dataset = HateSpeechDataset(texts)

    # Process in batches
    all_outputs = []
    for i in range(0, len(dataset), BATCH_SIZE):
        batch_texts = [dataset[j] for j in range(i, min(i + BATCH_SIZE, len(dataset)))]
        outputs = pipeline_fn(batch_texts, top_k=2)
        all_outputs.extend(outputs)

    # Convert to numpy array with consistent order
    return np.array([
        [label['score'] for label in sorted(res, key=lambda x: x['label'])]
        for res in all_outputs
    ])


# Model-specific prediction functions
def predict_cardiff(texts):
    return batch_predict(texts, pipe_cardiff)



# Initialize explainers
explainer_lime = LimeTextExplainer(random_state=42)
explainer_shap_cardiff = shap.Explainer(predict_cardiff,
                                       masker=shap.maskers.Text(tokenizer_cardiff))

# Helper function for LIME explanations
def lime_explain(text, predictor,explainer_lime=explainer_lime):
    exp = explainer_lime.explain_instance(text,
                                          predictor,
                                          )
    return ", ".join([f"{word}: {weight}" for word, weight in exp.as_list()])


# Helper function for SHAP explanations
def get_shap_values(text, explainer, class_idx=1):
    shap_vals = explainer([text])
    return ", ".join([
        f"{feature}: {value}"
        for feature, value in zip(shap_vals[0].data, shap_vals[0].values[:, class_idx])
        if abs(value) != 0
    ])


# Sample data
data = {
    'text': [
        "nawt yall niggers ignoring me",
        "and this is why i end up with nigger trainee doctors who can not speak properly lack basic knowledge of biology it truly scary if the public only knew",
        "this is a neutral sentence"
    ]
}
annotated_and_targeted_hatespeech_df = pd.DataFrame(data)


# Process all texts in one batch for efficiency
all_texts = annotated_and_targeted_hatespeech_df["text"].tolist()

# Get predictions in batch
cardiff_preds = pipe_cardiff(all_texts)

# Process results
results = []
for i, text in enumerate(all_texts):
    try:
        # Get predictions from the batch results
        pred_cardiff = cardiff_preds[i]

        # LIME explanations
        lime_exp_cardiff = lime_explain(text, predict_cardiff)

        # SHAP explanations
        shap_exp_cardiff = get_shap_values(text, explainer_shap_cardiff)

        # Store results
        results.append({
            "text": text,
            "error": None,
            "cardiff_label": pred_cardiff["label"],
            "cardiff_score": pred_cardiff["score"],
            "lime_cardiff": lime_exp_cardiff,
            "shap_cardiff": shap_exp_cardiff
        })
    except Exception as e:
        results.append({
            "text": text,
            "error": f"Caught an exception: {e}",
            "cardiff_label": None,
            "cardiff_score": None,
            "lime_cardiff": None,
            "shap_cardiff": None
        })

final_df = pd.DataFrame(results)


# Print the results
final_df.head()

Device set to use cuda:0


,text,error,cardiff_label,cardiff_score,lime_cardiff,shap_cardiff
0,nawt yall niggers ignoring me,None,HATE,0.935527,"niggers: -0.9069090692957803, nawt: -0.1023064...",": 5.4016709327697754e-08, n: 0.009972568834200..."
1,and this is why i end up with nigger trainee d...,None,HATE,0.993975,"nigger: -0.982060437234584, with: -0.005178025...",": 3.4924596548080444e-10, and : 0.002760904608..."
2,this is a neutral sentence,None,NOT-HATE,0.999157,"a: 0.0007440501285001838, is: 0.00068648281309...","this : 0.0007638931274414062, is : 0.000668436..."


In [22]:
# Saving to drive
from google.colab import drive
drive.mount('/content/drive')

# Save to your Google Drive
final_df.to_csv('/content/drive/MyDrive/hate_speech_analysis_results.csv', index=False)
final_df.to_excel('/content/drive/MyDrive/hate_speech_analysis_results.xlsx', index=False)
final_df.to_pickle('/content/drive/MyDrive/hate_speech_analysis_results.pkl')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import pandas as pd
import requests
from io import BytesIO

# GitHub raw content URL for the pickle file
github_raw_url = "https://github.com/Takosaga/master_thesis/raw/main/data/processed/experiement_samples.pkl"
# Download the pickle file
response = requests.get(github_raw_url)

# Check if the request was successful
if response.status_code == 200:
    # Load the pickle data into a pandas DataFrame
    df = pd.read_pickle(BytesIO(response.content))
    print(f"DataFrame loaded successfully with {len(df)} rows")

    # Display the first few rows
    print("\nPreview of the DataFrame:")
    display(df.head())
else:
    print(f"Failed to download the file. Status code: {response.status_code}")
    print(f"Response: {response.text}")

DataFrame loaded successfully with 652 rows

Preview of the DataFrame:


,text,extracted_target,platform,is_hatespeech,id,cardiff_score,cardiff_hatespeech
0,"India is one nasty, dirty, filthy, germsy, law...",{nationality},youtube,1,53513,0.994301,1
1,solo que seras bombardeas y destruidos como lo...,{religion},gab,1,8335,0.957913,1
2,@user @user You cannot argue with a retard jer...,"{disability, nationality}",twitter,1,64112,0.994864,1
3,Can I just pound that pussy an pull your hair....,{gender},reddit,1,22034,0.977588,1
4,you do not live in america nazi faggot but you...,{sexuality},gab,1,18143,0.985650,1


In [24]:
def process_dataframe(df, seed, text_column="text"):
    """Process an entire dataframe with efficient batching"""
    all_texts = df[text_column].tolist()
    total_texts = len(all_texts)

    # Get predictions in batch
    cardiff_preds = pipe_cardiff(all_texts)

    results = []
    for i, text in tqdm(enumerate(all_texts), total=total_texts, desc="Processing texts"):
        try:
            set_seed(seed)
            pred_cardiff = cardiff_preds[i]

            # Calculate explanations for all examples and setting seed
            set_seed(seed)
            explainer_lime = LimeTextExplainer(random_state=seed)
            lime_exp_cardiff = lime_explain(text, predict_cardiff)
            set_seed(seed)
            explainer_shap_cardiff = shap.Explainer(predict_cardiff,
                                       seed=seed,
                                       masker=shap.maskers.Text(tokenizer_cardiff))
            shap_exp_cardiff = get_shap_values(text, explainer_shap_cardiff)

            results.append({
                "text": text,
                "error": None,
                "cardiff_label": pred_cardiff["label"],
                "cardiff_score": pred_cardiff["score"],
                "lime_cardiff": lime_exp_cardiff,
                "shap_cardiff": shap_exp_cardiff,
            })
        except Exception as e:
            results.append({
                "text": text,
                "error": f"Caught an exception: {e}",
                "cardiff_label": None,
                "cardiff_score": None,
                "lime_cardiff": None,
                "shap_cardiff": None,
            })

    return pd.DataFrame(results)

In [25]:
def save_dataframe(df, name, base_path='/content/drive/MyDrive/'):
    """
    Save a dataframe to CSV, Excel, and pickle formats.

    Parameters:
    - df: pandas DataFrame to save
    - name: base name for the files (without extension)
    - base_path: directory path where files will be saved
    """
    # Save as CSV
    df.to_csv(f'{base_path}{name}.csv', index=False)

    # Save as Excel
    df.to_excel(f'{base_path}{name}.xlsx', index=False)

    # Save as pickle
    df.to_pickle(f'{base_path}{name}.pkl')

In [26]:
random_numbers = [random.randint(1, 100000) for _ in range(4)]

# Print the generated numbers
print("Generated random numbers:", random_numbers)

Generated random numbers: [83811, 14593, 3279, 97197]


In [27]:
random_numbers[1]

14593

In [28]:
f'df_rerun_{random_numbers[0]}'

'df_rerun_83811'

In [29]:
df_seed_first = process_dataframe(df, random_numbers[0])
save_dataframe(df_seed_first, f'df_rerun_{random_numbers[0]}')

Processing texts:   0%|          | 0/652 [00:00<?, ?it/s]

In [19]:
df_seed_second = process_dataframe(df, random_numbers[1])
save_dataframe(df_seed_second, f'df_rerun_{random_numbers[1]}')

Processing texts:   0%|          | 0/652 [00:00<?, ?it/s]

In [ ]:
df_seed_third = process_dataframe(df, random_numbers[2])
save_dataframe(df_seed_third, f'df_rerun_{random_numbers[2]}')

In [ ]:
df_seed_fourth = process_dataframe(df, random_numbers[3])
save_dataframe(df_seed_fourth, f'df_rerun_{random_numbers[3]}')